In [1]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorWithPadding,
)
import sys
from pathlib import Path
import datasets
import bitsandbytes as bnb
import loralib as lora
import numpy as np
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
sys.path.append(str(Path.cwd().parent))
from utils import CLASSIFICATION_PROMPT, remove_extra_brackets
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss, Linear
from tqdm import tqdm

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")
tokenizer.pad_token = tokenizer.eos_token
print(f"Model max length is {tokenizer.model_max_length} characters.")
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForSequenceClassification.from_pretrained(
    "Qwen/Qwen3-4B-Instruct-2507",
    device_map="cuda",
    num_labels=3,
    quantization_config=quantization_config,
    attn_implementation="sdpa",
    torch_dtype=torch.bfloat16,
)
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = model.config.eos_token_id

lora_config = LoraConfig(
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    modules_to_save=["score"],
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
lora.mark_only_lora_as_trainable(model)

`torch_dtype` is deprecated! Use `dtype` instead!


Model max length is 1010000 characters.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-4B-Instruct-2507 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.load_state_dict(torch.load("./best_qwen_llm_finetune_model.pth"), strict=True)

<All keys matched successfully>

In [4]:
model.save_pretrained("./finetuned_qwen_llm_model")

In [5]:
# Load multiple CSV files
df = datasets.load_dataset(
    "csv", data_files={"train": "../data/train.csv", "test": "../data/test.csv"}
)

In [6]:
df = df['train'].select(range(1000))

In [7]:
def fix_dataset(row):
    cleaned_prompt = remove_extra_brackets(row["prompt"])
    cleaned_response_a = remove_extra_brackets(row["response_a"])
    cleaned_response_b = remove_extra_brackets(row["response_b"])

    full_prompt = CLASSIFICATION_PROMPT.format(
        prompt=cleaned_prompt,
        response_a=cleaned_response_a,
        response_b=cleaned_response_b,
    )

    tokenized = tokenizer(full_prompt)

    winner = [row["winner_model_a"], row["winner_model_b"], row["winner_tie"]]

    return {**tokenized, "winner": winner, "length": len(tokenized["input_ids"])}

df = df.map(fix_dataset, batched=False).remove_columns(
    [
        "id",
        "model_a",
        "model_b",
        "prompt",
        "response_a",
        "response_b",
        "winner_model_a",
        "winner_model_b",
        "winner_tie",
    ]
)

df = df.filter(
    lambda batch: np.array(batch["length"]) <= 12000, batched=True
).remove_columns(["length"])

In [8]:
df = df.with_format("torch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [9]:
dataloader = DataLoader(
    df, batch_size=2, shuffle=False, collate_fn=data_collator
)

In [10]:
# loss_fn = CrossEntropyLoss()
# total_loss = 0
# total_correct = 0
# total_count = 0
# for data in tqdm(dataloader):
#     with torch.no_grad():
#         with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
#             data = {
#                 key: value.to("cuda") for key, value in data.items()
#             }
#             outputs = model(data["input_ids"]).logits
#             _, predicted = torch.max(outputs, 1)
#             _, true_labels = torch.max(data["winner"], 1)
#             total_loss += loss_fn(outputs, true_labels).item()
#             total_count += true_labels.size(0)
#             total_correct += (predicted == true_labels).sum().item()

# accuracy = 100 * (total_correct / total_count)
# print(f"Validation Accuracy: {accuracy:.2f}%")
# print(f"Validation Loss: {total_loss:.2f}%")

In [11]:
from huggingface_hub import notebook_login, HfApi

In [12]:
notebook_login()

In [13]:
# Merge LoRA weights into the base model
# model = model.merge_and_unload()

# Now push the full merged model
model.push_to_hub("Vmpletsos/qwen3-4b-merged-preference-classifier")
tokenizer.push_to_hub("Vmpletsos/qwen3-4b-merged-preference-classifier")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/Vmpletsos/qwen3-4b-merged-preference-classifier/commit/b8b89eb832405c75c41711468b68aa65df6384f6', commit_message='Upload tokenizer', commit_description='', oid='b8b89eb832405c75c41711468b68aa65df6384f6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Vmpletsos/qwen3-4b-merged-preference-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Vmpletsos/qwen3-4b-merged-preference-classifier'), pr_revision=None, pr_num=None)